In [1]:
import pandas as pd 
import numpy as np 
from data_reader import *
from evaluate_new import *
from sklearn_crfsuite import CRF 
from sklearn_crfsuite import metrics

/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)

Bad key "ckend" on line 1 in
/Users/sichenglei/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution
/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import re
## Get the doc level test data
doc_dir = "../../data_doc/"
def get_doc_test(labels="golden_data", text="test_file"):
    test_labels = []
    test_doc = []
    with open(doc_dir+labels, 'r') as doc_labels, open(doc_dir+text, 'r') as doc_text:
        d_labels = doc_labels.readlines()
        d_text = doc_text.readlines()
        assert len(d_labels) == len(d_text), "Mismatch"
        for i in range(len(d_labels)):
            test_labels.append(d_labels[i].strip())
            
            text = d_text[i].strip()
            text = re.sub('\d', '0', text)
            text = re.sub('[^ ]- ', '', text)
            
            test_doc.append(text)
    return test_labels, test_doc
            

In [3]:
doc_test_y, doc_test_x = get_doc_test()

In [4]:
def read_doc(doc, labels):
    doc = doc.strip().split()
    labels = labels.strip().split('|')
    labels = [la.split() for la in labels]
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            labels[i][j] = int(labels[i][j])

    res_labels = [0]*len(doc)
    for la in labels:
        if la[2]!=0:
            start = la[0]
            end = la[1]
            res_labels[start : end+1] = [1]*(end+1-start)
    return [(doc[i], str(res_labels[i])) for i in range(len(doc))]

In [6]:
## This script implements a purely feature based CRF.
train_sents = get_sents("../../data_30_0.1%neg/train.txt")
val_sents = get_sents("../../data_30_0.1%neg/validate.txt")
test_sents = get_sents("../../data_30_0.1%neg/test.txt")


def word2features(sent, i):
    word = sent[i][0]

    ##youmay add more features
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }

    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })

    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

##CRF takes string as labels
def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

##labels are strings
X_train = [sent2features(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]
X_test = [sent2features(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]



In [7]:
doc_tests = [read_doc(doc_test_x[d], doc_test_y[d]) for d in range(len(doc_test_x))]

In [8]:
X_doc_test = [sent2features(s) for s in doc_tests]


In [9]:
# predict one doc
def doc_pred(model, doc, MAXLEN):
    splits = []
    for i in range(0, len(doc), MAXLEN):
        splits.append(doc[i : i+MAXLEN])
    preds = model.predict(splits)
    preds = [p for pd in preds for p in pd]
    return preds

In [10]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

crf.fit(X_train, Y_train)

# labels = list(crf.classes_)
# y_pred = crf.predict(X_test)
# ##average F1
# # metrics.flat_f1_score(Y_test, y_pred,
# #                       average='weighted', labels=labels)

# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
# print(metrics.flat_classification_report(
#     Y_test, y_pred, labels=sorted_labels, digits=3
# ))


CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [18]:
y_pred = crf.predict(X_test)

In [4]:
y_pred = crf.predict(X_test)
preds = [[int(a) for a in x] for x in y_pred]
##record the prediceted start and end index
with open('../../outputs/CRF_preds', 'w') as fout:
    with open('../../data/test.txt', 'r') as test:
        test_list = test.readlines()
        for i in range(len(preds)):
            sent = test_list[i].strip().split()
            data_id = int(sent[2])
            pub_id = int(sent[3])

            first = 0
            j = 0
            string = ''
            no_mention = True
            while j<len(preds[i]):
                while j<len(preds[i]) and preds[i][j]== 0:
                    j+=1
                if j<len(preds[i]) and preds[i][j] == 1:
                    no_mention=False
                    start = j
                    while j+1<len(preds[i]) and preds[i][j+1]==1:
                        j+=1
                    end = j 
                    if first > 0:
                        string += " | "
                    string += (str(start)+' '+str(end)+' '+str(data_id)+' '+str(pub_id))
                    j+=1
                    first += 1
            if no_mention:
                fout.write("0 0 0 "+str(pub_id)+'\n')
            else:
                fout.write(string+'\n')


In [31]:
## doc level pred
doc_preds = [doc_pred(crf, d, 30) for d in X_doc_test]
doc_preds = [[int(a) for a in x] for x in doc_preds]

In [32]:
## record the prediceted start and end index
## for doc level
## write all start and end indices (0 0 if not mention)
with open('../../doc_outputs/CRF_preds', 'w') as fout:
    for i in range(len(doc_preds)):
        first = 0
        j = 0
        string = ''
        no_mention = True
        while j<len(doc_preds[i]):
            while j<len(doc_preds[i]) and doc_preds[i][j]== 0:
                j+=1
            if j<len(doc_preds[i]) and doc_preds[i][j] == 1:
                no_mention=False
                start = j
                while j+1<len(doc_preds[i]) and doc_preds[i][j+1]==1:
                    j+=1
                end = j 
                if first > 0:
                    string += " | "
                string += (str(start)+' '+str(end))
                j+=1
                first += 1
        if no_mention:
            fout.write("0 0"'\n')
        else:
            fout.write(string+'\n')


In [3]:
print (doc_exact_match('../../doc_outputs/CRF_preds', '../../data_doc/golden_data'))

0.0017406440382941688


In [4]:
print (doc_partial_match('../../doc_outputs/CRF_preds', '../../data_doc/golden_data'))

0.0034812880765883376


In [5]:
print (discovery_exact_match('../../outputs/CRF_preds', '../../data/test.txt'))

0.583873290136789


In [6]:
print (discovery_partial_match('../../outputs/CRF_preds', '../../data/test.txt'))

0.7930409568684306


In [10]:
import pandas as pd 
import numpy as np 
from data_reader import *
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.callbacks import EarlyStopping
from sklearn import metrics 
from keras_contrib.layers import CRF
import keras

Using TensorFlow backend.


In [21]:
# BLSTM
def sent2features(sent):
	return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
	return [int(label) for token, label in sent]

def sent2tokens(sent):
	return [token for token, label in sent]

X_train = [sent2tokens(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]
X_val = [sent2tokens(s) for s in val_sents]
Y_val = [sent2labels(s) for s in val_sents]
X_test = [sent2tokens(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]


# ##load glove
embedding_index = {}
f = open('../glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embedding_index[word] = coefs
f.close()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train+X_val+X_test)
word_index = tokenizer.word_index

##hyperparameters
vocab_size = len(word_index)+1
maxlen = 30
emb_dim = 50

embedding_matrix = np.zeros((vocab_size, emb_dim))
counter = 0
for word, i in word_index.items():
	embedding_vector = embedding_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
		counter += 1
	else:
		embedding_matrix[i] = np.random.randn(emb_dim)
print ("{}/{} words covered in glove".format(counter, vocab_size))

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)


X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

Y_train = np.asarray(Y_train)
Y_val = np.asarray(Y_val)
Y_test = np.asarray(Y_test)

#labels need to be 3D
Y_train = np.expand_dims(Y_train, axis=2)
Y_val = np.expand_dims(Y_val, axis=2)
Y_test = np.expand_dims(Y_test, axis=2)

##build model
input = Input(shape=(maxlen,))
model = Embedding(vocab_size, emb_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(1, activation='sigmoid'))(model)

model = Model(input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
earlyStop = [EarlyStopping(monitor='val_loss', patience=1)]
history = model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_data=(X_val, Y_val), 
	callbacks=earlyStop) 


# preds = model.predict(X_test)
# test = [[1 if y>=0.5 else 0 for y in x] for x in preds]
# test_arr = np.asarray(test)
# test = np.reshape(test_arr, (-1))
# target = np.reshape(Y_test, (-1))

# print (metrics.precision_recall_fscore_support(target, test, average=None,
#                                               labels=[0, 1]))


# preds = test_arr
# ##record the prediceted start and end index
# with open('../../outputs/BiLSTM_preds', 'w') as fout:
# 	with open('../../data/test.txt', 'r') as test:
# 		test_list = test.readlines()
# 		for i in range(len(preds)):
# 			sent = test_list[i].strip().split()
# 			data_id = int(sent[2])
# 			pub_id = int(sent[3])

# 			first = 0
# 			j = 0
# 			string = ''
# 			no_mention = True
# 			while j<len(preds[i]):
# 				while j<len(preds[i]) and preds[i][j]== 0:
# 					j+=1
# 				if j<len(preds[i]) and preds[i][j] == 1:
# 					no_mention=False
# 					start = j
# 					while j+1<len(preds[i]) and preds[i][j+1]==1:
# 						j+=1
# 					end = j 
# 					if first > 0:
# 						string += " | "
# 					string += (str(start)+' '+str(end)+' '+str(data_id)+' '+str(pub_id))
# 					j+=1
# 					first += 1
# 			if no_mention:
# 				fout.write("0 0 0 "+str(pub_id)+'\n')
# 			else:
# 				fout.write(string+'\n')


12785/18096 words covered in glove
Train on 24094 samples, validate on 3347 samples
Epoch 1/10
24094/24094 [==============================] - 36s 1ms/step - loss: 0.2253 - acc: 0.9127 - val_loss: 0.1625 - val_acc: 0.9344
Epoch 2/10
24094/24094 [==============================] - 33s 1ms/step - loss: 0.1538 - acc: 0.9396 - val_loss: 0.1368 - val_acc: 0.9461
Epoch 3/10
24094/24094 [==============================] - 34s 1ms/step - loss: 0.1339 - acc: 0.9474 - val_loss: 0.1239 - val_acc: 0.9511
Epoch 4/10
24094/24094 [==============================] - 34s 1ms/step - loss: 0.1226 - acc: 0.9518 - val_loss: 0.1140 - val_acc: 0.9547
Epoch 5/10
24094/24094 [==============================] - 33s 1ms/step - loss: 0.1143 - acc: 0.9551 - val_loss: 0.1066 - val_acc: 0.9580
Epoch 6/10
24094/24094 [==============================] - 33s 1ms/step - loss: 0.1078 - acc: 0.9573 - val_loss: 0.1069 - val_acc: 0.9569


In [7]:
print (discovery_exact_match('../../outputs/BiLSTM_preds', '../../data/test.txt'))
print (discovery_partial_match('../../outputs/BiLSTM_preds', '../../data/test.txt'))

0.5713324360699865
0.8089459084604715


In [25]:
X_doc_test = [sent2tokens(s) for s in doc_tests]

In [44]:
# predict one doc
def doc_pred(model, doc, MAXLEN):
    splits = []
    for i in range(0, len(doc), MAXLEN):
        splits.append(doc[i : i+MAXLEN])
    splits = tokenizer.texts_to_sequences(splits)
    splits = pad_sequences(splits, maxlen=MAXLEN)
    preds = model.predict(splits)
    
    preds = [1 if p>=0.5 else 0 for pd in preds for p in pd]
    return preds

In [45]:
doc_preds = [doc_pred(model, d, 30) for d in X_doc_test]

In [48]:
with open('../../doc_outputs/BiLSTM_preds', 'w') as fout:
    for i in range(len(doc_preds)):
        first = 0
        j = 0
        string = ''
        no_mention = True
        while j<len(doc_preds[i]):
            while j<len(doc_preds[i]) and doc_preds[i][j]== 0:
                j+=1
            if j<len(doc_preds[i]) and doc_preds[i][j] == 1:
                no_mention=False
                start = j
                while j+1<len(doc_preds[i]) and doc_preds[i][j+1]==1:
                    j+=1
                end = j 
                if first > 0:
                    string += " | "
                string += (str(start)+' '+str(end))
                j+=1
                first += 1
        if no_mention:
            fout.write("0 0"'\n')
        else:
            fout.write(string+'\n')


In [50]:
print (doc_partial_match('../../doc_outputs/BiLSTM_preds', '../../data_doc/golden_data'))
print (doc_exact_match('../../doc_outputs/BiLSTM_preds', '../../data_doc/golden_data'))

0.0003898635477582846
0.0


In [13]:
#This script implements a BiLSTM-CRF model
train_sents = get_sents("../../data_30_0.1%neg/train.txt")
val_sents = get_sents("../../data_30_0.1%neg/validate.txt")
test_sents = get_sents("../../data_30_0.1%neg/test.txt")


def sent2labels(sent):
	return [int(label) for token, label in sent]

def sent2tokens(sent):
	return [token for token, label in sent]

X_train = [sent2tokens(s) for s in train_sents]
Y_train = [sent2labels(s) for s in train_sents]
X_val = [sent2tokens(s) for s in val_sents]
Y_val = [sent2labels(s) for s in val_sents]
X_test = [sent2tokens(s) for s in test_sents]
Y_test = [sent2labels(s) for s in test_sents]


##load glove
embedding_index = {}
f = open('../glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embedding_index[word] = coefs
f.close()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train+X_val+X_test)
word_index = tokenizer.word_index

##hyperparameters
vocab_size = len(word_index)+1
maxlen = 30
emb_dim = 50

embedding_matrix = np.zeros((vocab_size, emb_dim))
counter = 0
for word, i in word_index.items():
	embedding_vector = embedding_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
		counter += 1
	else:
		embedding_matrix[i] = np.random.randn(emb_dim)
print ("{}/{} words covered in glove".format(counter, vocab_size))

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)


X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

Y_train = np.asarray(Y_train)
Y_val = np.asarray(Y_val)
Y_test = np.asarray(Y_test)

#labels need to be 3D
Y_train = np.expand_dims(Y_train, axis=2)
Y_val = np.expand_dims(Y_val, axis=2)
Y_test = np.expand_dims(Y_test, axis=2)

##build model
input = Input(shape=(maxlen,))
model = Embedding(vocab_size, emb_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False)(input)
model = Bidirectional(LSTM(100, return_sequences=True, recurrent_dropout=0.2))(model)
model = TimeDistributed(Dense(50, activation='relu'))(model)
model = TimeDistributed(Dropout(0.2))(model)
##use CRF instead of Dense
crf = CRF(2)
out = crf(model)

model = Model(input, out)


Y_train_2 = keras.utils.to_categorical(Y_train)
Y_val_2 = keras.utils.to_categorical(Y_val)
Y_test_2 = keras.utils.to_categorical(Y_test)

model.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy]) 
earlyStop = [EarlyStopping(monitor='val_loss', patience=1)]
history = model.fit(X_train, Y_train_2, batch_size=64, epochs=10, 
                   validation_data=(X_val, Y_val_2), callbacks=earlyStop)


preds = model.predict(X_test)
test = [[np.argmax(y) for y in x] for x in preds]
test_arr = np.asarray(test)
test = np.reshape(test_arr, (-1))

# print (metrics.precision_recall_fscore_support(np.reshape(Y_test,(-1)), test, average=None,
#                                               labels=[0, 1]))


# preds = test_arr
# ##record the prediceted start and end index
# with open('../../outputs/BiLSTM_CRF_preds', 'w') as fout:
# 	with open('../../data/test.txt', 'r') as test:
# 		test_list = test.readlines()
# 		for i in range(len(preds)):
# 			sent = test_list[i].strip().split()
# 			data_id = int(sent[2])
# 			pub_id = int(sent[3])

# 			first = 0
# 			j = 0
# 			string = ''
# 			no_mention = True
# 			while j<len(preds[i]):
# 				while j<len(preds[i]) and preds[i][j]== 0:
# 					j+=1
# 				if j<len(preds[i]) and preds[i][j] == 1:
# 					no_mention=False
# 					start = j
# 					while j+1<len(preds[i]) and preds[i][j+1]==1:
# 						j+=1
# 					end = j 
# 					if first > 0:
# 						string += " | "
# 					string += (str(start)+' '+str(end)+' '+str(data_id)+' '+str(pub_id))
# 					j+=1
# 					first += 1
# 			if no_mention:
# 				fout.write("0 0 0 "+str(pub_id)+'\n')
# 			else:
# 				fout.write(string+'\n')




12785/18096 words covered in glove
Train on 24094 samples, validate on 3347 samples
Epoch 1/10
24094/24094 [==============================] - 43s 2ms/step - loss: 0.1476 - acc: 0.9049 - val_loss: 0.0908 - val_acc: 0.9352
Epoch 2/10
24094/24094 [==============================] - 39s 2ms/step - loss: 0.0777 - acc: 0.9429 - val_loss: 0.0608 - val_acc: 0.9497
Epoch 3/10
24094/24094 [==============================] - 41s 2ms/step - loss: 0.0556 - acc: 0.9508 - val_loss: 0.0434 - val_acc: 0.9545
Epoch 4/10
24094/24094 [==============================] - 40s 2ms/step - loss: 0.0388 - acc: 0.9556 - val_loss: 0.0287 - val_acc: 0.9566
Epoch 5/10
24094/24094 [==============================] - 43s 2ms/step - loss: 0.0238 - acc: 0.9583 - val_loss: 0.0162 - val_acc: 0.9600
Epoch 6/10
24094/24094 [==============================] - 42s 2ms/step - loss: 0.0098 - acc: 0.9602 - val_loss: 0.0013 - val_acc: 0.9616
Epoch 7/10
24094/24094 [==============================] - 36s 1ms/step - loss: -0.0042 - acc: 

In [14]:
print (metrics.precision_recall_fscore_support(np.reshape(Y_test,(-1)), test, average=None,
                                              labels=[0, 1]))

(array([0.98683506, 0.8217574 ]), array([0.97159827, 0.90992774]), array([0.9791574 , 0.86359793]), array([70207, 10103]))


In [15]:
doc_preds = [doc_pred(model, d, 30) for d in X_doc_test]

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 183 arrays: [array([[{'bias': 1.0, 'word.lower()': 'copyright', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'BOS': True, '+1:word.lower()': '0000', '+1:word.istitle()': False, '+1:wo...

In [15]:
preds = test_arr
##record the prediceted start and end index
with open('../../doc_outputs/BiLSTM_CRF_preds', 'w') as fout:
	with open('../../data/test.txt', 'r') as test:
		test_list = test.readlines()
		for i in range(len(preds)):
			sent = test_list[i].strip().split()
			data_id = int(sent[2])
			pub_id = int(sent[3])

			first = 0
			j = 0
			string = ''
			no_mention = True
			while j<len(preds[i]):
				while j<len(preds[i]) and preds[i][j]== 0:
					j+=1
				if j<len(preds[i]) and preds[i][j] == 1:
					no_mention=False
					start = j
					while j+1<len(preds[i]) and preds[i][j+1]==1:
						j+=1
					end = j 
					if first > 0:
						string += " | "
					string += (str(start)+' '+str(end)+' '+str(data_id)+' '+str(pub_id))
					j+=1
					first += 1
			if no_mention:
				fout.write("0 0 0 "+str(pub_id)+'\n')
			else:
				fout.write(string+'\n')


In [8]:
print (discovery_exact_match('../../outputs/BiLSTM_CRF_preds', '../../data/test.txt'))
print (discovery_partial_match('../../outputs/BiLSTM_CRF_preds', '../../data/test.txt'))

0.6839745984310796
0.8752334703025775


In [21]:
import keras 
from keras import layers
from keras.layers import Embedding, Dropout, Dense, LSTM, Bidirectional, Input, Dense, Flatten
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_recall_fscore_support as score
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd 
import os

def data_loader(file):
	with open(file, 'r') as f:
		sents = []
		labels = []
		for line in f:
			line = line.strip().split()
			labels.append(int(line[2]))
			sents.append(' '.join(line[4:]))
	return sents, labels


train_sents, train_labels = data_loader("../../data/train.txt")
val_sents, val_labels = data_loader("../../data/validate.txt")
test_sents, test_labels = data_loader("../../data/test.txt")


data_set = pd.read_json('../../../train_test/data_sets.json', encoding='utf-8')
# #note: dataset_id = index + 1
data_description = data_set["description"].values


# Add a sentence for no mention case
data_description = list(data_description)
data_description.insert(0, "There is no mention.")


maxlen = 30
#vocab_size = 40000 ##more than 80K unique tokens
emb_dim = 50
HIDDEN_DIM = 256
EPOCHS = 10  
NEG_RATIO = 3
BATCH_SIZE = 10
DATASET_CLASS = len(data_description) 
MODEL_NAME = "CNN"

#actual batch size = BATCH_SIZE * (1 + NEG_RATIO)

##load glove
# embedding_index = {}
# f = open('../glove.6B.50d.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = np.asarray(values[1:], dtype='float32')
# 	embedding_index[word] = coefs
# f.close()


tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sents+val_sents+test_sents)
X_train = tokenizer.texts_to_sequences(train_sents)
X_val = tokenizer.texts_to_sequences(val_sents)
X_test = tokenizer.texts_to_sequences(test_sents)

word_index = tokenizer.word_index
##hyperparameters
vocab_size = len(word_index)+1

embedding_matrix = np.zeros((vocab_size, emb_dim))
counter = 0
for word, i in word_index.items():
	embedding_vector = embedding_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
		counter += 1
	else:
		embedding_matrix[i] = np.random.randn(emb_dim)
print ("{}/{} words covered in glove".format(counter, vocab_size))



X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
Y_train = np.asarray(train_labels)
Y_val = np.asarray(val_labels)
Y_test = np.asarray(test_labels)
Y_train = keras.utils.to_categorical(Y_train, num_classes=DATASET_CLASS)
Y_val = keras.utils.to_categorical(Y_val, num_classes=DATASET_CLASS)
Y_test = keras.utils.to_categorical(Y_test, num_classes=DATASET_CLASS)


##randomly shuffle data and labels
##np.random.seed(0)
N = X_train.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)
X_train = X_train[indices]
Y_train = Y_train[indices] 


def build_model():
	#CNN model, 1D conv
	model = Sequential()
	model.add(layers.Embedding(vocab_size, emb_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
	model.add(layers.Conv1D(64, 5, activation='relu'))
	model.add(layers.MaxPooling1D(3))
	model.add(layers.Conv1D(32, 4, activation='relu'))
	model.add(layers.GlobalMaxPooling1D())
	model.add(layers.Dense(DATASET_CLASS, activation="sigmoid"))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model 

model = build_model()

callbacks_list = [
	keras.callbacks.EarlyStopping(
		monitor='val_acc',
		patience=2)
]

history = model.fit(X_train, Y_train, 
					epochs=EPOCHS,
					batch_size=128,
					validation_data=(X_val, Y_val),
					callbacks=callbacks_list)



print ('CNN model test accuracy: ', model.evaluate(X_test, Y_test))



12760/17811 words covered in glove
Train on 24094 samples, validate on 3347 samples
Epoch 1/10
24094/24094 [==============================] - 21s 867us/step - loss: 5.8249 - acc: 0.1100 - val_loss: 4.9291 - val_acc: 0.1183
Epoch 2/10
24094/24094 [==============================] - 17s 693us/step - loss: 4.7587 - acc: 0.1151 - val_loss: 4.5548 - val_acc: 0.1183
Epoch 3/10
2677/2677 [==============================] - 1s 320us/step
CNN model test accuracy:  [4.345270836936657, 0.11206574525112174]


In [22]:
Y_test.shape

(2677, 10349)

In [23]:
preds = model.predict(X_test)

In [26]:
Y2_test = np.asarray([np.argmax(a) for a in Y_test])
preds2 = np.asarray([np.argmax(a) for a in preds])

In [38]:
import sklearn
print (sklearn.metrics.precision_recall_fscore_support(Y2_test, preds2, average='micro'))
print (sklearn.metrics.precision_recall_fscore_support(Y2_test, preds2, average='macro'))

(0.11206574523720583, 0.11206574523720583, 0.11206574523720583, None)
(0.00018898102063609753, 0.0016863406408094434, 0.0003398738275061022, None)


/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
import keras 
from keras import layers
from keras.layers import Embedding, Dropout, Dense, LSTM, Bidirectional, Input, Dense, Flatten
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_recall_fscore_support as score
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd 
import os

def data_loader(file):
    with open(file, 'r') as f:
        sents = []
        labels = []
        for line in f:
            line = line.strip().split()
            labels.append(int(line[2]))
            sents.append(' '.join(line[4:]))
    return sents, labels


train_sents, train_labels = data_loader("../../data/train.txt")
val_sents, val_labels = data_loader("../../data/validate.txt")
test_sents, test_labels = data_loader("../../data/test.txt")


data_set = pd.read_json('../../../train_test/data_sets.json', encoding='utf-8')
#note: dataset_id = index + 1
data_description = data_set["description"].values


# Add a sentence for no mention case
data_description = list(data_description)
data_description.insert(0, "There is no mention.")


maxlen = 30
#vocab_size = 40000 ##more than 80K unique tokens
emb_dim = 50
HIDDEN_DIM = 256
EPOCHS = 10  ## train more epochs with GPU, it takes 1h per epoch on my CPU
NEG_RATIO = 3
BATCH_SIZE = 10
DATASET_CLASS = len(data_description) 
MODEL_NAME = "LSTM"

#actual batch size = BATCH_SIZE * (1 + NEG_RATIO)

##load glove
embedding_index = {}
f = open('../glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

###NOT using dataset info anymore
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_sents+val_sents+test_sents)
X_train = tokenizer.texts_to_sequences(train_sents)
X_val = tokenizer.texts_to_sequences(val_sents)
X_test = tokenizer.texts_to_sequences(test_sents)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

embedding_matrix = np.zeros((vocab_size, emb_dim))
counter = 0
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        counter += 1
    else:
        embedding_matrix[i] = np.random.randn(emb_dim)
print ("{}/{} words covered in glove".format(counter, vocab_size))

X_train = pad_sequences(X_train, maxlen=maxlen)
X_val = pad_sequences(X_val, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
Y_train = np.asarray(train_labels)
Y_val = np.asarray(val_labels)
Y_test = np.asarray(test_labels)
Y_train = keras.utils.to_categorical(Y_train, num_classes=DATASET_CLASS)
Y_val = keras.utils.to_categorical(Y_val, num_classes=DATASET_CLASS)
Y_test = keras.utils.to_categorical(Y_test, num_classes=DATASET_CLASS)


##randomly shuffle data and labels
##np.random.seed(0)
N = X_train.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)
X_train = X_train[indices]
Y_train = Y_train[indices] 


def build_model():
    embedding_layer = Embedding(vocab_size, emb_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False)
    article_input = Input(shape=(maxlen,), dtype='int32')
    article_emb = embedding_layer(article_input)
    
    article_lstm = LSTM(HIDDEN_DIM, dropout=0.2, recurrent_dropout=0.3)
    article_vector = article_lstm(article_emb)
    #vector shape: (batch_size, hidden_dim)
    
    dense_vector = Dense(HIDDEN_DIM*4)(article_vector)
    
    dense_vector = Dropout(0.3)(dense_vector)
    output = Dense(DATASET_CLASS, activation='sigmoid')(dense_vector)
    
    model = Model(article_input, output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 

model = build_model()

callbacks_list = [
	keras.callbacks.EarlyStopping(
		monitor='val_acc',
		patience=2)
]

history = model.fit(X_train, Y_train, 
                    epochs=EPOCHS,
                    batch_size=128,
                    validation_data=(X_val, Y_val),
                    callbacks=callbacks_list)


print ('BiLSTM test accuracy: ', model.evaluate(X_test, Y_test))





12760/17811 words covered in glove
Train on 24094 samples, validate on 3347 samples
Epoch 1/10
24094/24094 [==============================] - 125s 5ms/step - loss: 5.6160 - acc: 0.1044 - val_loss: 4.8284 - val_acc: 0.1183
Epoch 2/10
24094/24094 [==============================] - 114s 5ms/step - loss: 4.6441 - acc: 0.1150 - val_loss: 4.4153 - val_acc: 0.1183
Epoch 3/10
24094/24094 [==============================] - 118s 5ms/step - loss: 4.1616 - acc: 0.1202 - val_loss: 3.8129 - val_acc: 0.1365
Epoch 4/10
24094/24094 [==============================] - 115s 5ms/step - loss: 3.5787 - acc: 0.2073 - val_loss: 3.2048 - val_acc: 0.2689
Epoch 5/10
24094/24094 [==============================] - 132s 5ms/step - loss: 3.0829 - acc: 0.2773 - val_loss: 2.8075 - val_acc: 0.3209
Epoch 6/10
24094/24094 [==============================] - 135s 6ms/step - loss: 2.7288 - acc: 0.3259 - val_loss: 2.5817 - val_acc: 0.3448
Epoch 7/10
24094/24094 [==============================] - 133s 6ms/step - loss: 2.4907 -

In [43]:
preds = model.predict(X_test)
Y2_test = np.asarray([np.argmax(a) for a in Y_test])
preds2 = np.asarray([np.argmax(a) for a in preds])
print (sklearn.metrics.precision_recall_fscore_support(Y2_test, preds2, average='micro'))
print (sklearn.metrics.precision_recall_fscore_support(Y2_test, preds2, average='macro'))

(0.38924168845722823, 0.38924168845722823, 0.38924168845722823, None)
(0.14152911254941078, 0.153139380196953, 0.13664545465552758, None)


/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/sichenglei/Desktop/kaggle/salt/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
